In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
import glob
import pafy
import youtube_dl
import io
from PIL import Image
from tqdm import tqdm_notebook, tqdm
import cv2
import json
from tqdm import tqdm_notebook as tqdm
from pathlib import Path
import os
import math
import datetime
from google.protobuf.json_format import MessageToJson
import ast


#from google.cloud import videointelligence_v1p3beta1 as videointelligence
from google.cloud import videointelligence
from google.oauth2 import service_account

In [12]:
video_url = "https://www.youtube.com/watch?v=I84eb-B0l94"

video_title = '2femix89pTE'

#video_uri = 'gs://videos-detection/videos_files/F1/2020 Eifel Grand Prix/2020 Eifel Grand Prix.mkv'

path_data = Path('../data/')
path_json = path_data / 'logo-detection.json'
pathIn = path_data / str(video_title + '/input/')
pathIn_Video = pathIn / str(video_title + '.mp4')
pathIn_Frames =  pathIn / 'frames'
pathIn_Frames_Resized = pathIn / 'frames_resized'

pathOut = path_data / str(video_title + '/output/')
path_results= pathOut / 'result.json'
path_annotations = pathOut / 'annotations.csv'
path_logos_video = pathOut / 'Logos.mp4'

In [8]:
import urllib
def get_video_id(video_url):
    url_data = urllib.parse.urlparse(video_url)
    query = urllib.parse.parse_qs(url_data.query)
    return query["v"][0]

'I84eb-B0l94'

In [7]:
import io
with io.open(pathIn_Video, 'rb') as f:
    input_video = f.read()

In [14]:
ii = str(input_video).encode('utf-8')
ii[:100]

b"b'\\x00\\x00\\x00 ftypisom\\x00\\x00\\x02\\x00isomiso2avc1mp41\\x00\\x00\\x00\\x08free\\x02\\xea\\x82,mdat\\x00\\x00"

In [7]:
credentials = service_account.Credentials.from_service_account_file(path_json)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = str(path_json)

In [18]:
def create_paths():

    if not os.path.exists(pathOut):
        os.makedirs(pathOut)

    if not os.path.exists(pathIn_Frames):
        os.makedirs(pathIn_Frames)
        
    if not os.path.exists(pathIn_Frames_Resized):
        os.makedirs(pathIn_Frames_Resized)
    
def load_videos(video_file):
    
    capture = cv2.VideoCapture(video_file)

    fps = capture.get(cv2.CAP_PROP_FPS)
    
    read_flag, frame = capture.read()
    
    vid_frames = []

    while (read_flag):
        
        vid_frames.append(frame)
        
        read_flag, frame = capture.read()
        
    vid_frames = np.asarray(vid_frames, dtype='uint8')
    
    capture.release()
    
    return vid_frames, fps

def annotate_video(input_video, local=False):
    """
    Performs asynchronous video annotation for logo recognition on a local file.

    """
    client = videointelligence.VideoIntelligenceServiceClient(credentials=credentials)

    if local:
        with io.open(input_video, 'rb') as f:
            input_video = f.read()
        
    features = [videointelligence.enums.Feature.LOGO_RECOGNITION]
    operation = client.annotate_video(input_video, features=features)
    print('\nProcessing video for logo annotations:')
    
    print(u'Waiting for operation to complete...')
    response = operation.result(timeout=1000)
    print('\nFinished processing.')
  
    return response

def video_to_frames(vid_input, pathOut):

    #start the video
    
    fourcc = cv.VideoWriter_fourcc(*'XVID')
    out = cv.VideoWriter('output.avi', fourcc, 20.0, (640,  480))

    cap = cv2.VideoCapture(str(vid_input))

    if (cap.isOpened()== False): 
        print("Error opening video stream or file")

    count = 0

    while(cap.isOpened()):

        #Capture frame-by-frame
        ret, frame = cap.read()

        if ret==True:

            write_frame(count, frame, pathOut)

            ret, frame = cap.read()
            count += 1
            wait = cv2.waitKey(10)

            if wait==27 & 0xFF == ord('q'):
                break   

        else:
            break

    cap.release()
    cv2.destroyAllWindows()


def write_frame(count, frame, pathOut):

    cv2.imwrite(str(pathOut / f'frame{count}.jpg'), frame)

    if not cv2.imwrite(str(pathOut / f'frame{count}.jpg'), frame):
        raise Exception("Could not write image")
        
def download_video(video_url, pathIn_Video):
    """
    Download a video using youtube url and video title
    """

    ydl_opts = {'outtmpl': str(pathIn_Video)}
    
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])
        
def download_video_2(video_url, pathIn_Video):
    """
    Download a video using youtube url and video title
    """
    from pytube import YouTube
    yt = YouTube(video_url)
    yt = yt.get('mp4', '720p')
    yt.download(str(pathIn_Video))

        
def anno_to_pandas(pathIn_Video, result):

    vcap = cv2.VideoCapture(str(pathIn_Video))

    fps = vcap.get(cv2.CAP_PROP_FPS)
    width, height = (int(vcap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(vcap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    annotation_result = result['annotationResults'][0]
    
    DfLogos = pd.DataFrame()

    for label in tqdm(annotation_result['logoRecognitionAnnotations']):
        for track in label['tracks']:
            confidence = track['confidence']
            for idx, anno in enumerate(track['timestampedObjects']):
                
                micro, seconds = math.modf(float(anno['timeOffset'].replace('s', '')))
                bounding_box = dict()
                anno_bounding_box = anno['normalizedBoundingBox']
                for key in ['top', 'bottom', 'left', 'right']:
                    bounding_box[key] = anno_bounding_box.get(key, 0)

                df_tmp = pd.DataFrame({'seconds': seconds,
                                       'micro': micro,
                                       'frame': int((seconds + micro)*fps/2)+1,
                                       'logo': label['entity']['description'],
                                       'x1': int(bounding_box['left']*width), 
                                       'y1': int(bounding_box['top']*height),
                                       'x2': int(bounding_box['right']*width),
                                       'y2': int(bounding_box['top']*height),
                                       'x3': int(bounding_box['right']*width),
                                       'y3': int(bounding_box['bottom']*height),
                                       'x4': int(bounding_box['left']*width),
                                       'y4': int(bounding_box['bottom']*height),
                                       'conficence': confidence}, index=[idx])
                DfLogos = DfLogos.append(df_tmp, sort=False, ignore_index=True)

    return DfLogos

def video_url_to_frames(video_url, pathOut):

    ydl_opts = {}

    # create youtube-dl object
    ydl = youtube_dl.YoutubeDL(ydl_opts)

    # set video url, extract video information
    info_dict = ydl.extract_info(video_url, download=False)

    # get video formats available
    formats = info_dict.get('formats',None)

    for f in formats:

        # I want the lowest resolution, so I set resolution as 144p
        if f.get('format_note',None) == '144p':

            #get the video url
            url = f.get('url',None)
            
            cap = cv2.VideoCapture(url)

            if (cap.isOpened()== False): 
                print("Error opening video stream or file")

            count = 0

            while(cap.isOpened()):

                #Capture frame-by-frame
                ret, frame = cap.read()

                if ret==True:

                    write_frame(count, frame, pathOut)

                    ret, frame = cap.read()
                    count += 1
                    wait = cv2.waitKey(10)

                    if wait==27 & 0xFF == ord('q'):
                        break   

                else:
                    break
            

#             # open url with opencv
#             cap = cv2.VideoCapture(url)

#             # check if url was opened
#             if not cap.isOpened():
#                 print('video not opened')
#                 exit(-1)

#             ret, frame = cap.read()
#             count = 0
#             while True:
                
#                 write_frame(count, frame, pathOut)
                    
#                 # read frame
#                 ret, frame = cap.read()
                
#                 count += 1
                
#                 # check if frame is empty
#                 if not ret:
#                     break

#                 # display frame
#                 cv2.imshow('frame', frame)

#                 if cv2.waitKey(30)&0xFF == ord('q'):
#                     break

            # release VideoCapture
            cap.release()

    cv2.destroyAllWindows()
    

In [5]:
from IPython.display import YouTubeVideo
YouTubeVideo(video_url)

In [143]:
create_paths()

In [ ]:
download_video(video_url, pathIn_Video)

In [11]:
#download_video_2(video_url, pathIn_Video)

In [8]:
# aca habria q llamar a una funcion que suba el video a cloud storage

In [1]:
result = annotate_video(pathIn_Video, True)
#result = annotate_video(video_uri)

NameError: name 'annotate_video' is not defined

In [163]:
#video_to_frames(pathIn_Video, pathIn_Frames)
video_url_to_frames(video_url, pathIn_Frames)

[youtube] 2femix89pTE: Downloading webpage
[youtube] 2femix89pTE: Downloading video info webpage


In [145]:
def persist_result(result, path_results):
    serialized = MessageToJson(result)
    serialized = ast.literal_eval(serialized)
    with open(str(path_results), 'w') as f:
        json.dump(serialized, f)

In [146]:
persist_result(result, path_results)

In [10]:
def load_result(path_results):
    with open(str(path_results), 'r') as data:
        results = json.loads(data.read())
    return results

In [13]:
results = load_result(path_results)

In [15]:
results['annotationResults']

[{'inputUri': '/videos-detection/videos_files/F1/2femix89pTE/video.mp4',
  'error': {'code': 15,
   'message': 'Calculator failed, commonly caused by invalid video, or unsupported format, or file not found, or permission denied.'}}]

In [157]:
# url = generate_image_url('F1 Hamilton.mp4', 'videos-detection')
# req.urlretrieve(url, "F1 Hamilton.mp4")

DfLogos = anno_to_pandas(pathIn_Video, results)
DfLogos = DfLogos.sort_values(by='frame')
DfLogos.head()

/Users/julian/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:128: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


,seconds,micro,frame,logo,x1,y1,x2,y2,x3,y3,x4,y4,conficence
968,0.0,0.1,2,Grupo Radio Centro,698,437,739,437,739,483,698,483,0.882708
969,0.0,0.2,3,Grupo Radio Centro,701,440,742,440,742,486,701,486,0.882708
970,0.0,0.3,4,Grupo Radio Centro,703,444,745,444,745,490,703,490,0.882708
477,0.0,0.3,4,Red Bull,1665,552,1719,552,1719,627,1665,627,0.917755
491,0.0,0.4,6,Red Bull,284,325,425,325,425,423,284,423,0.952237


In [158]:
DfLogos.to_csv(str(path_annotations))